In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime
import pandas as pd
import re

headers = {
 'sec-ch-ua':'"Chromium";v="136", "Google Chrome";v="136", "Not.A/Brand";v="99"',
 'sec-ch-ua-mobile':'?0',
 'sec-ch-ua-platform':'"macOS"',
 'upgrade-insecure-requests':'1',
 'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36',
 'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
 'sec-fetch-site':'none',
 'sec-fetch-mode':'navigate',
 'sec-fetch-user':'?1',
 'sec-fetch-dest':'document',
 'accept-encoding':'gzip, deflate, br, zstd',
 'accept-language':'zh-CN,zh;q=0.9',
 'priority':'u=0, i',
 'cookie':'HWWAFSESID=1495b887d9db23f80d; HWWAFSESTIME=1748060487534; VerificationToken=xGotHRYwnpov36TAIu+P6Z+11q8FfVnmPtn1y1rhb6JfUBCDw81uzI9WKvI15rt5hULwI6lhRTGpHwZjjbvpgeQA7jFj+yz876ECNwpUYwCRacZNLi/rvGkn9/xlTe4h/eidU6gQ2ybzYRW8+f8Z9K6BqkNNlZQfpQ5fjcWJhw4=; Hm_lvt_78958f7e120af53474ceac97f8d686b8=1748071968; HMACCOUNT=357320963718B80D; Hm_lvt_78958f7e120af53474ceac97f8d686b8=1748071968; HMACCOUNT=357320963718B80D; _pk_id.33.95f4=9aea3a99f1a9f98e.1748071981.; _pk_id.33.95f4=9aea3a99f1a9f98e.1748071981.; _pk_ses.33.95f4=1; ASP.NET_SessionId=0mpwo3ywncpexhabwjcxg5id; updateusermenu=5750216b-c88f-482c-8163-aee87d9eea5c; __RequestVerificationToken=d-s_ivR6v8kBuUXgsZeaUzGM9_TZZ5soi5wgD5Qtq7DIPaSuLaWhtgxK3F8ASYiz_t-VboV5dmKY-BU8ipn0Ojfw-Jk1; faeASwe=1; User=id=e910119165c3725b&name=e2bab48e88903bf2&key1=241543e7ad382de8&key2=7745750e8babb0b5; VerifyMessage1=9A7C5EB4F7BBF30915DD46643C7E7CB2; faeASweP=1; rUrl=https://www.dzmyy.com.cn/Account/Resetting; .ASPXAUTH.dzm_gb=6ECEFABB039CA84520E6730EDE184312A02BBB91BCB17491D22B64610B57237135721E01AC36EB4401ED69B245235236F62F824BD52D13B677A8465BC208871D5723D500B6BDD68E2D4562ADEB8D0EFCD538B6943C7026BF4FB3650EC98B04CA42A6DCBEE908743FBB08C4CCE20E0930AA1427FA0C13FA854AFB551BEFE9B4873C580FEB9D19E5AC7530648A1458F3E8C46BBF95D39361DE906D7BC7BCDC6FB966220067; _pk_ses.33.95f4=1; first_day_Christmas=10cooking_Fish=958D961BE98FF43C06D57649E5132DE25148A306AF76C358&10fool_When=2|1141072372&9cooking_Fish=958D961BE98FF43C06D57649E5132DE25148A306AF76C358&9fool_When=2|1160645136&8cooking_Fish=958D961BE98FF43C06D57649E5132DE25148A306AF76C358&8fool_When=2|1959050299&7cooking_Fish=958D961BE98FF43C06D57649E5132DE25148A306AF76C358&7fool_When=2|-1932049114&6cooking_Fish=958D961BE98FF43C06D57649E5132DE25148A306AF76C358&6fool_When=2|-0681805872&5cooking_Fish=958D961BE98FF43C06D57649E5132DE25148A306AF76C358&5fool_When=2|-0480118226&4cooking_Fish=958D961BE98FF43C06D57649E5132DE25148A306AF76C358&4fool_When=2|0985904299&3cooking_Fish=958D961BE98FF43C06D57649E5132DE25148A306AF76C358&3fool_When=2|2010862864&2cooking_Fish=958D961BE98FF43C06D57649E5132DE25148A306AF76C358&2fool_When=2|-2085782784&1cooking_Fish=958D961BE98FF43C06D57649E5132DE25148A306AF76C358&1fool_When=2|0074354220; Hm_lpvt_78958f7e120af53474ceac97f8d686b8=1748076616; Hm_lpvt_78958f7e120af53474ceac97f8d686b8=1748076616'}
payload=None

response0 = requests.request("GET", "https://www.dzmyy.com.cn/Interactions/SchedulingAppointments/OPDoctorIndex?OPdepartmentId=270", headers=headers, data=payload)


In [2]:
from bs4 import BeautifulSoup

def extract_available_appointments_doctor(html_str, doctor_name="袁玲玲"):
    # 解析HTML
    soup = BeautifulSoup(html_str, 'lxml')
    table = soup.find('table')
    
    # 存储可预约的条目
    available_entries = []
    
    # 获取表头（th标签）作为字段名
    headers = [th.get_text(strip=True) for th in table.find('tr').find_all('th')]
    
    # 查找"医生姓名"在表头中的索引位置
    try:
        doctor_index = headers.index('医生姓名')
    except ValueError:
        print("警告: 表格中未找到'医生姓名'列")
        return available_entries
    
    # 遍历每一行（跳过表头）
    for row in table.find_all('tr')[1:]:
        cells = row.find_all('td')
        if not cells:  # 跳过空行
            continue
            
        # 提取每列数据
        data = {headers[i]: cells[i].get_text(strip=True) for i in range(len(headers))}
        
        # 检查医生姓名是否匹配
        if data.get('医生姓名') != doctor_name:
            continue
        
        # 检查操作列是否有符合条件的链接
        operation_cell = cells[-1]  # 操作列是最后一列
        link = operation_cell.find('a', class_='dailySchedulingState_a_4')
        
        if link and '/Interactions/SchedulingAppointments/OPCreate' in link.get('href', ''):
            # 提取预约链接
            appointment_url = "https://www.dzmyy.com.cn" + link.get('href')
            data['预约链接'] = appointment_url
            
            # 添加到结果列表
            available_entries.append(data)
    
    return available_entries

In [3]:
def extract_available_appointments_time(html_str):
    # 解析HTML
    soup = BeautifulSoup(html_str, 'lxml')
    table = soup.find('table')
    available_entries = []
    
    return available_entries

In [4]:
# 待解析的HTML字符串（请替换为你的实际数据）
available_appointments_doctor = extract_available_appointments_doctor(response0.text, "袁玲玲")
for entry_doctor in available_appointments_doctor:
    print(f"{entry_doctor['就诊时间']} {entry_doctor['时段']} - {entry_doctor['医生姓名']} ({entry_doctor['出诊级别']})")
    print(f"预约链接: {entry_doctor['预约链接']}")
    print("-" * 50)

2025-06-03 上午 - 袁玲玲 (副主任医师)
预约链接: https://www.dzmyy.com.cn/Interactions/SchedulingAppointments/OPCreate?dailySchedulingId=2115301
--------------------------------------------------
2025-06-03 下午 - 袁玲玲 (副主任医师)
预约链接: https://www.dzmyy.com.cn/Interactions/SchedulingAppointments/OPCreate?dailySchedulingId=2115400
--------------------------------------------------
2025-06-04 上午 - 袁玲玲 (副主任医师)
预约链接: https://www.dzmyy.com.cn/Interactions/SchedulingAppointments/OPCreate?dailySchedulingId=2115694
--------------------------------------------------
2025-06-05 上午 - 袁玲玲 (副主任医师)
预约链接: https://www.dzmyy.com.cn/Interactions/SchedulingAppointments/OPCreate?dailySchedulingId=2116192
--------------------------------------------------
2025-06-09 上午 - 袁玲玲 (副主任医师)
预约链接: https://www.dzmyy.com.cn/Interactions/SchedulingAppointments/OPCreate?dailySchedulingId=2118028
--------------------------------------------------
2025-06-09 下午 - 袁玲玲 (副主任医师)
预约链接: https://www.dzmyy.com.cn/Interactions/SchedulingAppointments/

In [ ]:
url = 'https://www.dzmyy.com.cn/Interactions/SchedulingAppointments/OPCreate?dailySchedulingId=2115301'
response0 = requests.request("GET", url, headers=headers, data=payload, allow_redirects=False)

print(response0.headers['Location'])

response1 = requests.request("GET", response0.headers['Location'], headers=headers, data=payload)

print(response0.history)
print(response0.text)


/Account/LogOn?returnUrl=/Interactions/SchedulingAppointments/OPCreate?dailySchedulingId=2115301
[]
<html><head><title>Object moved</title></head><body>
<h2>Object moved to <a href="/Account/LogOn?returnUrl=/Interactions/SchedulingAppointments/OPCreate?dailySchedulingId=2115301">here</a>.</h2>
</body></html>

